# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import ClusterCentroids
from imblearn.combine import SMOTEENN

# Read the CSV into DataFrame

In [3]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df_lending_data = pd.read_csv(file_path)
df_lending_data.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


# Split the Data into Training and Testing

In [4]:
# Create our features
X = df_lending_data.copy()
X.drop("loan_status", axis=1, inplace=True)

#We would need to encode the data for it be able to scale correctly.  Without enconding we can get error of converting STR to INT
# Fitting and encoding the columns with the LabelEncoder
# Create the LabelEncoder instance
le = LabelEncoder()

# Encode Home Ownership column
le.fit(X["homeowner"])
X["homeowner"] = le.transform(X["homeowner"])
X.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,1,52800,0.431818,5,1,22800
1,8400.0,6.692,1,43600,0.311927,3,0,13600
2,9000.0,6.963,2,46100,0.349241,3,0,16100
3,10700.0,7.664,1,52700,0.430740,5,1,22700
4,10800.0,7.698,0,53000,0.433962,5,1,23000


In [5]:
# Create our target
y = df_lending_data["loan_status"]
y 

0         low_risk
1         low_risk
2         low_risk
3         low_risk
4         low_risk
           ...    
77531    high_risk
77532    high_risk
77533    high_risk
77534    high_risk
77535    high_risk
Name: loan_status, Length: 77536, dtype: object

In [6]:
X.describe()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,0.606144,49221.949804,0.377318,3.826610,0.392308,19221.949804
std,2093.223153,0.889495,0.667811,8371.635077,0.081519,1.904426,0.582086,8371.635077
min,5000.000000,5.250000,0.000000,30000.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,0.000000,44800.000000,0.330357,3.000000,0.000000,14800.000000
50%,9500.000000,7.172000,1.000000,48100.000000,0.376299,4.000000,0.000000,18100.000000
75%,10400.000000,7.528000,1.000000,51400.000000,0.416342,4.000000,1.000000,21400.000000
max,23800.000000,13.235000,2.000000,105200.000000,0.714829,16.000000,3.000000,75200.000000


In [7]:
y

0         low_risk
1         low_risk
2         low_risk
3         low_risk
4         low_risk
           ...    
77531    high_risk
77532    high_risk
77533    high_risk
77534    high_risk
77535    high_risk
Name: loan_status, Length: 77536, dtype: object

In [8]:
# Check the balance of our target values
Counter(y)

Counter({'low_risk': 75036, 'high_risk': 2500})

In [9]:
# Create X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
Counter(y_train)

Counter({'low_risk': 56271, 'high_risk': 1881})

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [10]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [11]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [12]:
# Scale the training and testing data

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Simple Logistic Regression

In [13]:

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [14]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9520479254722232

In [15]:
def confusion_matrix_df(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
    )
    return cm_df

In [16]:
# Display the confusion matrix

display(confusion_matrix_df(y_test, y_pred))

,Predicted 0,Predicted 1
Actual 0,563,56
Actual 1,102,18663


In [17]:
# Print the imbalanced classification report

print(classification_report_imbalanced(y_test, y_pred, digits=10))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk  0.8466165414 0.9095315024 0.9945643485 0.8769470405 0.9510981054 0.8968956403       619
   low_risk  0.9970083872 0.9945643485 0.9095315024 0.9957848682 0.9510981054 0.9122795720     18765

avg / total  0.9922058412 0.9918489476 0.9122469034 0.9919899541 0.9510981054 0.9117883084     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [18]:
# Resample the training data with the RandomOversampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)


# View the count of target classes with Counter
Counter(y_resampled)

Counter({'low_risk': 56271, 'high_risk': 56271})

In [19]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [20]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9936781215845847

In [21]:
# Display the confusion matrix
display(confusion_matrix_df(y_test, y_pred))

,Predicted 0,Predicted 1
Actual 0,615,4
Actual 1,116,18649


In [22]:
# Print the imbalanced classification report

print(classification_report_imbalanced(y_test, y_pred, digits=10))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk  0.8413132695 0.9935379645 0.9938182787 0.9111111111 0.9936781117 0.9873685115       619
   low_risk  0.9997855573 0.9938182787 0.9935379645 0.9967929873 0.9936781117 0.9874238678     18765

avg / total  0.9947249740 0.9938093273 0.9935469159 0.9940568606 0.9936781117 0.9874221001     19384



### SMOTE Oversampling

In [23]:
# Resample the training data with SMOTE
X_resampled_smote, y_resampled_smote = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train, y_train
)

# View the count of target classes with Counter
Counter(y_resampled_smote)

Counter({'low_risk': 56271, 'high_risk': 56271})

In [24]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled_smote, y_resampled_smote)

LogisticRegression(random_state=1)

In [25]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9936781215845847

In [26]:
# Display the confusion matrix
display(confusion_matrix_df(y_test, y_pred))

,Predicted 0,Predicted 1
Actual 0,615,4
Actual 1,116,18649


In [27]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred, digits=10))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk  0.8413132695 0.9935379645 0.9938182787 0.9111111111 0.9936781117 0.9873685115       619
   low_risk  0.9997855573 0.9938182787 0.9935379645 0.9967929873 0.9936781117 0.9874238678     18765

avg / total  0.9947249740 0.9938093273 0.9935469159 0.9940568606 0.9936781117 0.9874221001     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [28]:
# Resample the data using the ClusterCentroids resampler
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'high_risk': 1881, 'low_risk': 1881})

In [29]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [30]:
# Calculate the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9865149130022852

In [31]:
# Display the confusion matrix
display(confusion_matrix_df(y_test, y_pred))

,Predicted 0,Predicted 1
Actual 0,606,13
Actual 1,112,18653


In [32]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred,digits=10))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk  0.8440111421 0.9789983845 0.9940314415 0.9065071055 0.9864862773 0.9716922257       619
   low_risk  0.9993035466 0.9940314415 0.9789983845 0.9966605220 0.9864862773 0.9746181251     18765

avg / total  0.9943445083 0.9935513826 0.9794784434 0.9937816031 0.9864862773 0.9745246907     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [33]:
# Resample the training data with SMOTEENN
smote_enn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)


# View the count of target classes with Counter
Counter(y_resampled)

Counter({'high_risk': 73880, 'low_risk': 74596})

In [34]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [35]:
# Calculate the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9935715401830394

In [36]:
# Display the confusion matrix
display(confusion_matrix_df(y_test, y_pred))

,Predicted 0,Predicted 1
Actual 0,615,4
Actual 1,120,18645


In [38]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred, digits=10))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk  0.8367346939 0.9935379645 0.9936051159 0.9084194978 0.9935715396 0.9871777752       619
   low_risk  0.9997855113 0.9936051159 0.9935379645 0.9966857326 0.9935715396 0.9871910334     18765

avg / total  0.9945787193 0.9936029715 0.9935401088 0.9938670781 0.9935715396 0.9871906100     19384



# Final Questions

1. Which model had the best balanced accuracy score?

   Naive Random Oversampling and SMOTE Oversampling have best balance accuracy score of 0.9936781215845847.

2. Which model had the best recall score?

   The Simple Logistic Regression model has the best recall value for "low_risk" value as it has true postive of 18663 and false negatives of 102 which gives the best recall value of all models which is 0.9945643485.  However in terms of overall recall value, it is  Naive Random Oversampling and SMOTE Oversampling which has the overall recall value of 0.9938093273. 

3. Which model had the best geometric mean score?

    Again for this one also Naive Random Oversampling and SMOTE Oversampling has the best geometric mean of 0.9940568606
